In [1]:
import gym
import numpy as np
import tensorflow as tf
from collections import deque

# Create the Pendulum environment
env = gym.make('Pendulum-v0')

# Define hyperparameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
hidden_size = 32
learning_rate = 0.001

# Define the DQN class
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = self._build_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_size, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(hidden_size, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.uniform(-2., 2., self.action_size)  # Random action within action space
        act_values = self.model.predict(state)
        return act_values[0]

    def replay(self, batch_size):
        minibatch = np.array(random.sample(self.memory, batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize DQN agent
agent = DQN(state_size, action_size)

# Training the agent
batch_size = 32
episodes = 1000

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):  # Maximum of 500 timesteps per episode
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = np.abs(reward)  # Modify reward if necessary
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"episode: {e+1}/{episodes}, score: {time}")
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
